### Conversion:
from senate_meta_v1.csv -> rows-2.json

This will match the FusionTables query response expected by the app

In [6]:
import json
import pandas as pd
import xml
import xml.etree.ElementTree as ET

In [15]:
df = pd.read_csv('../data/senate_meta_v1.csv')

##### app relies on this order

In [18]:
df = df.sort_values(by='senateId')

##### order to match app's query

In [19]:
query_cols = "MaxLong, poly, name, senateId, incomeShared, numOrgs, LatCenter, LongCenter, LatLength, LongLength"
query_cols = query_cols.split(', ')

df2 = df[query_cols]

##### convert shape to rows based array

In [20]:
d = {}
for k in df2.keys():
    d[k] = df2[k].to_list()
    
d_len = len(d[list(d.keys())[0]])

rows = []
for i in range(d_len):
    _row = [d[k][i] for k in list(d.keys())]
    rows.append(_row)

##### convert district polygon xml to json style

In [21]:
def parse_scoords(e):
    try:
        return [
            float(e.split(',')[0]), 
            float(e.split(',')[1])
        ]
    except:
        return None
    
def parseXml(s):
    
    s = s.replace('\r\n', '\n')
    
    root = ET.fromstring(s)
    
    s_coords = root[0][0][0]

    l_coords = s_coords.text.split('\n')

    ll_coords = [parse_scoords(e) for e in l_coords]
    ll_coords = [e for e in ll_coords if e is not None]
    
    d_geo = {
        'geometry': {
            'type': 'Polygon',
            'coordinates': [ll_coords]
        }
    }
    return d_geo

In [22]:
for i in range(d_len):
    rows[i][1] = parseXml(rows[i][1])

In [23]:
    
out_rows = {'rows': rows}

In [24]:
json.dump(out_rows, open('output/rows-4.json', 'w'))

##### Now copy to file to `app/data/` directory